<a href="https://colab.research.google.com/github/pacozaa/data-playground-notebook/blob/main/StickyNotes_Generator_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq datasets huggingface_hub

In [2]:
!git clone https://github.com/pacozaa/Category-Seed.git
%cd Category-Seed

fatal: destination path 'Category-Seed' already exists and is not an empty directory.
/content/Category-Seed


In [3]:
import os

# Print the current working directory
print("Current Working Directory:", os.getcwd())

# List all files and directories in the current working directory
print("Files and Directories in Current Directory:")
for item in os.listdir():
    print(item)

# Now, try to access the 'Category-Seed' directory
directory_path = './Category-Seed'
if os.path.exists(directory_path) and os.path.isdir(directory_path):
    print(f"Directory '{directory_path}' exists.")
else:
    print(f"Directory '{directory_path}' does not exist.")

Current Working Directory: /content/Category-Seed
Files and Directories in Current Directory:
Category-Seed
bash-agent.txt
company2.json
full-stack.txt
web-game.txt
.git
bash-category.txt
company4.json
README.md
unique_company.json
company3.json
merged_data.jsonl
company.json
Directory './Category-Seed' exists.


In [4]:
import os
import json

def read_json_files_in_directory(directory):
    # List to store the contents of all JSON files
    json_data = []

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        # Check if the file is a JSON file
        if filename.endswith('.json'):
            # Construct the full file path
            file_path = os.path.join(directory, filename)

            # Open and read the JSON file
            with open(file_path, 'r') as file:
                try:
                    # Load the JSON data
                    data = json.load(file)
                    # Append the data to the list
                    json_data.append(data)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in file {file_path}: {e}")

    return json_data

def write_jsonl_file(data, output_file):
    with open(output_file, 'w') as file:
        for item in data:
            for entry in item:
                json.dump(entry, file)
                file.write('\n')

# Example usage
directory_path = '/content/Category-Seed'
output_file = '/content/merged_data.jsonl'
all_json_data = read_json_files_in_directory(directory_path)

# Iterate through each JSON data
for i, data in enumerate(all_json_data):
    transformed_data = []
    for item in data:
        if 'companies' in item:
            # Iterate through each company in the 'companies' list
            for company in item['companies']:
                # Append the company name to the transformed data list
                transformed_data.append({'name': company['name'],'description': company['description']})
        else:
            transformed_data=data
    # Replace the original data with the transformed data
    all_json_data[i] = transformed_data


# Print the updated contents of all JSON data
for data in all_json_data:
    print(data)

# Write the merged data to a JSONL file
write_jsonl_file(all_json_data, output_file)

[{'name': 'Apple Inc.', 'description': 'Consumer electronics and software company.'}, {'name': 'Google LLC', 'description': 'Technology company specializing in Internet-related services and products.'}, {'name': 'Microsoft Corporation', 'description': 'Software development, manufacturing, and licensing.'}, {'name': 'Amazon.com Inc.', 'description': 'E-commerce and cloud computing company.'}, {'name': 'Facebook, Inc.', 'description': 'Social media and technology company.'}, {'name': 'Tesla, Inc.', 'description': 'Electric vehicle manufacturer and clean energy company.'}, {'name': 'Netflix, Inc.', 'description': 'Subscription-based streaming service and production company.'}, {'name': 'Airbnb, Inc.', 'description': 'Online marketplace for lodging and tourism activities.'}, {'name': 'Uber Technologies, Inc.', 'description': 'Ride-hailing and food delivery company.'}, {'name': 'SpaceX', 'description': 'Aerospace manufacturer and space transportation company.'}, {'name': 'Twitter, Inc.', 'd

In [7]:
from datasets import load_dataset
from huggingface_hub import login
from google.colab import userdata
if False:
  # Load the JSONL file into a Hugging Face dataset
  dataset = load_dataset('json', data_files='/content/merged_data.jsonl')

  # Print the dataset to verify
  print(dataset)

  # Authenticate with your Hugging Face token
  # login(token=userdata.get('hftoken'))

  # Push the dataset to the Hugging Face Hub
  dataset.push_to_hub("pacozaa/company-v1",token=userdata.get('hftoken'))

  print("Dataset has been pushed to the Hugging Face Hub.")

DatasetDict({
    train: Dataset({
        features: ['name', 'description', 'field'],
        num_rows: 754
    })
})


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset has been pushed to the Hugging Face Hub.


In [ ]:
# Load dataset
dataset = load_dataset("pacozaa/company-v1")
# map to find field using llm

In [9]:
from groq import Groq
client = Groq(
      api_key=userdata.get('GROQ_API_KEY')
  )

In [ ]:
def llm(prompt):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": f"""{prompt}""",
          }
      ],
      model="llama3-8b-8192",
  )
  return chat_completion.choices[0].message.content.strip()

In [ ]:
llm("hi")